In [1]:
%load_ext nb_black
# https://medium.com/openplanetary/code-formatting-in-jupyter-cells-8fee4eda072f

<IPython.core.display.Javascript object>

In [51]:
from skimage.draw import circle_perimeter
from skimage.segmentation import slic
from skimage.segmentation import mark_boundaries
from skimage.util import img_as_float
from skimage import io
import xml.etree.ElementTree as ET
import skimage.segmentation as seg
import skimage.filters as filt
import skimage.color as color
import matplotlib.pyplot as plt
import argparse
import json
import numpy as np
import matplotlib.patches as mpatches
from skimage.measure import label, regionprops
import math
import skimage.morphology as morph
import pandas as pd
import os
import shapely.geometry as shgeo
import mmcv

from classes.WheatDataset import WheatDataset
from helpers import utils

%matplotlib inline

<IPython.core.display.Javascript object>

In [3]:
from jinja2 import Environment, FileSystemLoader


class Writer:
    def __init__(self, path, width, height, depth=3, database="Unknown", segmented=0):
        environment = Environment(
            loader=FileSystemLoader("pascal_voc_writer_rotated/templates/")
        )
        self.annotation_template = environment.get_template("annotation.xml")

        abspath = os.path.abspath(path)

        self.template_parameters = {
            "path": abspath,
            "filename": os.path.basename(abspath),
            "folder": os.path.basename(os.path.dirname(abspath)),
            "width": width,
            "height": height,
            "depth": depth,
            "database": database,
            "segmented": segmented,
            "objects": [],
        }

    def addObject(
        self,
        object_type,
        name,
        cx,
        cy,
        w,
        h,
        angle,
        pose="Unspecified",
        truncated=0,
        difficult=0,
    ):
        self.template_parameters["objects"].append(
            {
                "type": object_type,
                "name": name,
                "cx": cx,
                "cy": cy,
                "w": w,
                "h": h,
                "angle": angle,
                "pose": pose,
                "truncated": truncated,
                "difficult": difficult,
            }
        )

    def save(self, annotation_path):
        with open(annotation_path, "w") as file:
            content = self.annotation_template.render(**self.template_parameters)
            file.write(content)

<IPython.core.display.Javascript object>

In [4]:
## move to utils


def dist(plot1, plot2):
    return math.sqrt((plot1[0] - plot2[0]) ** 2 + (plot1[1] - plot2[1]) ** 2)


def line(p1, p2):
    A = p1[1] - p2[1]
    B = p2[0] - p1[0]
    C = p1[0] * p2[1] - p2[0] * p1[1]
    return A, B, -C


def intersection(L1, L2):
    D = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x, y
    else:
        return False


def get_intersection(a1, a2, b1, b2):
    L1 = line(a1, a2)
    L2 = line(b1, b2)

    R = intersection(L1, L2)
    if R:
        #         print(R)
        return R
    else:
        print("No single intersection point detected")
        return None


# def get_corner_b_l(theta_rad, w, h, c):
#     """Returns bottom left corner of the box
#        (b_l)----- ----> x
#        |        | |
#        |        | |
#        ---------- y

#     Parameters
#     ----------
#     theta_rad : number
#         The file location of the spreadsheet
#     w : number
#         width of the box
#     h : number
#         height of the box
#     c : list [x, y]
#         center of the box

#     Returns
#     -------
#     number
#         bottom left corner of the box
#     """
#     w = w / 2
#     h = h / 2
#     c = np.array(c)
#     r = np.array(
#         (
#             (np.cos(theta_rad), -np.sin(theta_rad)),
#             (np.sin(theta_rad), np.cos(theta_rad)),
#         )
#     )

#     v = np.array((-w, -h))

#     return c + r.dot(v)


def cwha_to_xyp(x_c, y_c, w, h, a):
    """Returns four corners of the box
       First determine four axis aligned corner points
       (-w/2, -h/2)-----------(w/2, -h/2)
                   |         |
                   |    c    |
                   |         |
       (-w/2, h/2) -----------(w/2, h/2)
       
       Then, apply angle on it to get final corner points
       
       src * H = dst
    ...
    
    Parameters
    ----------
    x_c : number
          x coordinate of center
    y_c : number
          y coordinate of center
    w   : number
          width of the box
    h   : number
          height of the box
    a   : number
          angle of rotation in degrees

    Returns
    -------
    numpy array
        four corners of the box
    """
    w = w / 2
    h = h / 2
    c = np.array([[x_c, y_c]])
    a = math.radians(a)

    rotation = np.array([[np.cos(a), -np.sin(a)], [np.sin(a), np.cos(a)],])

    axis_aligned_box = np.array(
        [
            [-w, -h],  # bottom-left
            [w, -h],  # bottom-right
            [w, h],  # top-right
            [-w, h],  # top-left
        ]
    )

    return c + axis_aligned_box.dot(rotation.T)


def cwha_to_xyxy(x_c, y_c, w, h, a):
    poly = cwha_to_xyp(x_c, y_c, w, h, a)

    xmin = max(int(poly[:, 0].min()), 0)
    xmax = int(poly[:, 0].max())

    ymin = max(int(poly[:, 1].min()), 0)
    ymax = int(poly[:, 1].max())

    return [xmin, ymin, xmax, ymax]


def show_four_corners(image, corners):
    fig, ax = plt.subplots(1, 1, figsize=(10, 10))
    ax.imshow(image)
    colors = ["yellow", "red", "green", "blue"]

    for i, c in enumerate(corners):
        circ = mpatches.Circle((c[0], c[1]), 5, color=colors[i])
        ax.add_patch(circ)

    plt.tight_layout()
    plt.show()

<IPython.core.display.Javascript object>

In [5]:
def detect_orientation(img, img_g, box):
    global cnt, sm_boxes

    img_g = img_g > filt.threshold_otsu(img_g)
    img_g = morph.remove_small_objects(img_g)
    img_g = morph.remove_small_holes(img_g)
    img_g = morph.binary_opening(img_g, morph.disk(1))

    label_img = label(img_g)

    #     utils.display_image(label_img)

    regions = regionprops(label_img)

    prop = None
    mn = 0
    for props in regions:
        if props.area > mn:
            mn = props.area
            prop = props

    h, w = img_g.shape
    if mn < 15 or w < 5 or h < 5:
        cnt = cnt + 1
        sm_boxes.append(box)

        return []

    y0, x0 = prop.centroid

    x1 = x0 + math.cos(prop.orientation) * 0.5 * prop.major_axis_length
    y1 = y0 - math.sin(prop.orientation) * 0.5 * prop.major_axis_length
    x2 = x0 - math.sin(prop.orientation) * 0.5 * prop.minor_axis_length
    y2 = y0 - math.cos(prop.orientation) * 0.5 * prop.minor_axis_length

    #     fig, ax = plt.subplots()
    #     ax.imshow(img, cmap=plt.cm.gray)

    #     ax.plot((x0, x1), (y0, y1), "-r", linewidth=2.5)
    #     ax.plot((x0, x2), (y0, y2), "-y", linewidth=2.5)
    #     ax.plot(x0, y0, ".g", markersize=15)

    #     minr, minc, maxr, maxc = prop.bbox
    #     bx = (minc, maxc, maxc, minc, minc)
    #     by = (minr, minr, maxr, maxr, minr)

    #     ax.plot(bx, by, "-b", linewidth=2.5)
    #     plt.show()

    intersections = []
    intersections.append(get_intersection([x0, y0], [x1, y1], [0, 0], [w, 0]))
    intersections.append(get_intersection([x0, y0], [x1, y1], [0, 0], [0, h]))
    intersections.append(get_intersection([x0, y0], [x1, y1], [w, 0], [w, h]))
    intersections.append(get_intersection([x0, y0], [x1, y1], [0, h], [w, h]))

    points = []
    for i in intersections:
        if i[0] >= 0 and i[0] - w <= 0.0001 and i[1] >= 0 and i[1] - h <= 0.0001:
            points.append(i)

    if len(points) == 2:
        b_width = dist(points[0], points[1])
        b_height = prop.minor_axis_length + 10

        x_center = box[0] + 0.5 * abs(points[0][0] + points[1][0])
        y_center = box[1] + 0.5 * abs(points[0][1] + points[1][1])

        #         print(np.rad2deg(prop.orientation))

        ## considering positive y axis is downwards
        #         maj_ang = np.rad2deg(prop.orientation) + 90
        #         maj_ang = -(maj_ang - 180) if maj_ang > 90 else 180 - maj_ang
        #         print(maj_ang)

        # angle is from 0 to 180 clockwise from x axis
        maj_ang = np.rad2deg(prop.orientation)
        maj_ang = 180 - maj_ang if maj_ang > 0 else -maj_ang

        return [x_center, y_center, b_width, b_height, maj_ang]
    else:
        print(w, h)
        print(intersections)
        print(points)

        cnt = cnt + 1
        sm_boxes.append(box)

        fig, ax = plt.subplots()
        ax.imshow(img, cmap=plt.cm.gray)

        ax.plot((x0, x1), (y0, y1), "-r", linewidth=2.5)
        ax.plot((x0, x2), (y0, y2), "-y", linewidth=2.5)
        ax.plot(x0, y0, ".g", markersize=15)

        minr, minc, maxr, maxc = prop.bbox
        bx = (minc, maxc, maxc, minc, minc)
        by = (minr, minr, maxr, maxr, minr)

        ax.plot(bx, by, "-b", linewidth=2.5)
        plt.show()

        print("NOT FOUND")
        return []

<IPython.core.display.Javascript object>

In [6]:
def get_orientation(image, bbox):
    patch = image[bbox[1] : bbox[3], bbox[0] : bbox[2]]
    patch_g = color.rgb2gray(patch)
    patch_g_filt = filt.gaussian(patch_g, 1)

    #     if indx == 8:
    #     utils.display_image(patch, False, False)
    #     utils.display_image(patch_g > filt.threshold_otsu(patch_g), False, False)
    #     utils.display_image(patch_g_filt > filt.threshold_otsu(patch_g_filt), False, False)

    return detect_orientation(patch.copy(), patch_g_filt.copy(), bbox)

<IPython.core.display.Javascript object>

In [7]:
def detect_oriented_boxes(filename, bboxes, r_format="XYP"):
    image = utils.get_rgb_image(
        img_as_float(io.imread(os.path.join(path_images, filename)))
    )

    bboxes_xyp = utils.xyxy_to_poly(bboxes)

    rows = []
    rows_xyp = []
    for i in range(0, min(len(bboxes), 100)):
        cwha = get_orientation(image, bboxes[i])
        #         print(cwha)
        if cwha:
            xyp = cwha_to_xyp(*cwha).flatten()
            xyp[xyp < 0] = 0

            rows.append(xyp if r_format == "XYP" else cwha)
            rows_xyp.append(xyp)
            #             show_four_corners(image, list(xyp.reshape((-1, 2))))

    #     utils.show_poly_anno(
    #         os.path.join(path_images, filename),
    #         [np.array(rows_xyp), bboxes_xyp],
    #         ["Rotated", "Original"],
    #         False,
    #         (15, 15),
    #     )

    return rows

<IPython.core.display.Javascript object>

### Detect rotated bounding boxes per image and create a csv

#### Define some paths

In [84]:
path_images = "/home/badhon/Documents/thesis/Data and Scripts from AL/global-WHEAT-final/train/usas_minipam/"
path_splits = "/home/badhon/Documents/thesis/AerialDetection/data/wheat_competition/"
dataset_type = "test"
path_xml = "/home/badhon/Documents/thesis/rotated_annotation/" + dataset_type + "/"
path_axis_aligned_annotation_vis = os.path.join(
    "/home/badhon/Documents/thesis/axis_aligned_annotation_vis", dataset_type
)
path_rotated_annotation_vis = os.path.join(
    "/home/badhon/Documents/thesis/rotated_annotation_vis", dataset_type
)

<IPython.core.display.Javascript object>

#### Load Dataset

In [85]:
dataset = WheatDataset(
    os.path.join(path_splits, dataset_type + "/" + dataset_type + ".json"), path_images
)
dataset.load_dataset()

<IPython.core.display.Javascript object>

#### Some Variables

In [86]:
global cnt, sm_boxes

cnt = 0
sm_boxes = []

filenames = dataset.get_file_names()

<IPython.core.display.Javascript object>

#### Generate rotated bounding boxes annotations and save as xml

In [87]:
df_rows = []
for i, filename in enumerate(filenames[:]):
    print(str(i) + ":  " + filename)

    bboxes = np.array(dataset.get_bboxes_of_image(filename))
    r_bboxes = detect_oriented_boxes(filename, bboxes, r_format="CWHA")

    writer = Writer(
        os.path.join(path_splits, dataset_type + "/images", filename), "1024", "1024"
    )
    for b in r_bboxes:
        b[4] = math.radians(b[4])
        writer.addObject("robndbox", "wheat_head", *b)

    writer.save(path_xml + filename[:-4] + ".xml")

0:  crop_1_crop_0_905-compressed.png
1:  crop_1_crop_0_4745-compressed.png


/home/badhon/anaconda3/lib/python3.7/site-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/home/badhon/anaconda3/lib/python3.7/site-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


2:  crop_1_crop_0_473-compressed.png
3:  crop_1_crop_0_964-compressed.png
4:  crop_1_crop_0_4760-compressed.png
5:  crop_1_crop_0_4955-compressed.png
6:  crop_1_crop_0_698-compressed.png
7:  crop_1_crop_0_82-compressed.png
8:  crop_1_crop_0_967-compressed.png
9:  crop_1_crop_0_913-compressed.png
10:  crop_1_crop_0_98-compressed.png
11:  crop_1_crop_0_96-compressed.png
12:  crop_1_crop_0_710-compressed.png
13:  crop_1_crop_0_4909-compressed.png
14:  crop_1_crop_0_771-compressed.png
15:  crop_1_crop_0_521-compressed.png
16:  crop_1_crop_0_747-compressed.png
17:  crop_1_crop_0_810-compressed.png
18:  crop_1_crop_0_834-compressed.png
19:  crop_1_crop_0_4855-compressed.png


<IPython.core.display.Javascript object>

#### Save axis aligned annotations as images

In [88]:
for i, filename in enumerate(filenames[:]):
    print(str(i) + ":  " + filename)

    bboxes = np.array(dataset.get_bboxes_of_image(filename, r_format="XYP"))
    utils.show_poly_anno(
        os.path.join(path_images, filename),
        [bboxes],
        ["Original"],
        False,
        (20, 20),
        True,
        os.path.join(path_axis_aligned_annotation_vis, filename),
    )

0:  crop_1_crop_0_905-compressed.png
1:  crop_1_crop_0_4745-compressed.png
2:  crop_1_crop_0_473-compressed.png
3:  crop_1_crop_0_964-compressed.png
4:  crop_1_crop_0_4760-compressed.png
5:  crop_1_crop_0_4955-compressed.png
6:  crop_1_crop_0_698-compressed.png
7:  crop_1_crop_0_82-compressed.png
8:  crop_1_crop_0_967-compressed.png
9:  crop_1_crop_0_913-compressed.png
10:  crop_1_crop_0_98-compressed.png
11:  crop_1_crop_0_96-compressed.png
12:  crop_1_crop_0_710-compressed.png
13:  crop_1_crop_0_4909-compressed.png
14:  crop_1_crop_0_771-compressed.png
15:  crop_1_crop_0_521-compressed.png
16:  crop_1_crop_0_747-compressed.png
17:  crop_1_crop_0_810-compressed.png
18:  crop_1_crop_0_834-compressed.png
19:  crop_1_crop_0_4855-compressed.png


<IPython.core.display.Javascript object>

##### Save refined rotated annotations as images

In [94]:
def read_xml_rotated(xml_file: str, isSave=False):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []

    filename = root.find("filename").text + ".png"
    print(filename)

    for boxes in root.iter("object"):

        cx, cy, w, h, a = None, None, None, None, None

        for box in boxes.findall("robndbox"):
            cx = float(box.find("cx").text)
            cy = float(box.find("cy").text)
            w = float(box.find("w").text)
            h = float(box.find("h").text)
            a = float(box.find("angle").text)

            angle_deg = math.degrees(a)

            if w < h:
                temp = w
                w = h
                h = temp
                angle_deg = (angle_deg + 90) % 180

            list_with_single_boxes = [cx, cy, w, h, angle_deg]
            list_with_all_boxes.append(list_with_single_boxes)

    xyps = []
    for cwha in list_with_all_boxes:
        xyp = cwha_to_xyp(*cwha).flatten()
        xyps.append(xyp)

    xyps = np.array(xyps)

    if isSave:
        image = utils.get_rgb_image(
            img_as_float(io.imread(os.path.join(path_images, filename)))
        )

        utils.show_poly_anno(
            os.path.join(path_images, filename),
            [np.array(xyps)],
            ["Rotated"],
            False,
            (15, 15),
            True,
            os.path.join(path_rotated_annotation_vis, filename),
        )

    return filename, np.array(list_with_all_boxes), xyps

<IPython.core.display.Javascript object>

In [95]:
for file in os.listdir(path_xml):
    name, boxes, xyps = read_xml_rotated(os.path.join(path_xml, file), True)

crop_1_crop_0_4745-compressed.png
crop_1_crop_0_473-compressed.png
crop_1_crop_0_913-compressed.png
crop_1_crop_0_747-compressed.png
crop_1_crop_0_967-compressed.png
crop_1_crop_0_96-compressed.png
crop_1_crop_0_4909-compressed.png
crop_1_crop_0_98-compressed.png
crop_1_crop_0_4760-compressed.png
crop_1_crop_0_834-compressed.png
crop_1_crop_0_82-compressed.png
crop_1_crop_0_698-compressed.png
crop_1_crop_0_4855-compressed.png
crop_1_crop_0_4955-compressed.png
crop_1_crop_0_710-compressed.png
crop_1_crop_0_810-compressed.png
crop_1_crop_0_964-compressed.png
crop_1_crop_0_521-compressed.png
crop_1_crop_0_771-compressed.png
crop_1_crop_0_905-compressed.png


<IPython.core.display.Javascript object>

#### Save rotated annotations as json and csv

In [96]:
fname_im_id = dataset.get_file_name_image_id_pair()

<IPython.core.display.Javascript object>

In [97]:
data_dict = {
    "images": list(dataset._images.values()),
    "categories": list(dataset._categories.values()),
    "annotations": [],
}

<IPython.core.display.Javascript object>

In [98]:
inst_count = 1
anns = []
df_rows = []

for file in filenames[:]:
    name, b_cwha, b_xyp = read_xml_rotated(os.path.join(path_xml, file[:-4] + ".xml"))

    for i, p in enumerate(b_xyp[:]):

        df_rows.append([file, *b_cwha[i].astype(int), *p.astype(int)])

        poly = list(map(tuple, temp.reshape(-1, 2)))

        gtpoly = shgeo.Polygon(poly)
        poly = list(map(int, p))

        xmin, ymin, xmax, ymax = (
            min(poly[0::2]),
            min(poly[1::2]),
            max(poly[0::2]),
            max(poly[1::2]),
        )

        width, height = xmax - xmin, ymax - ymin

        single_obj = {}
        single_obj["category_id"] = 1
        single_obj["image_id"] = fname_im_id[file]
        single_obj["id"] = inst_count
        single_obj["area"] = int(gtpoly.area)
        single_obj["segmentation"] = [poly]
        single_obj["iscrowd"] = 0
        single_obj["bbox"] = [xmin, ymin, width, height]

        data_dict["annotations"].append(single_obj)
        inst_count = inst_count + 1

crop_1_crop_0_905-compressed.png
crop_1_crop_0_4745-compressed.png
crop_1_crop_0_473-compressed.png
crop_1_crop_0_964-compressed.png
crop_1_crop_0_4760-compressed.png
crop_1_crop_0_4955-compressed.png
crop_1_crop_0_698-compressed.png
crop_1_crop_0_82-compressed.png
crop_1_crop_0_967-compressed.png
crop_1_crop_0_913-compressed.png
crop_1_crop_0_98-compressed.png
crop_1_crop_0_96-compressed.png
crop_1_crop_0_710-compressed.png
crop_1_crop_0_4909-compressed.png
crop_1_crop_0_771-compressed.png
crop_1_crop_0_521-compressed.png
crop_1_crop_0_747-compressed.png
crop_1_crop_0_810-compressed.png
crop_1_crop_0_834-compressed.png
crop_1_crop_0_4855-compressed.png


<IPython.core.display.Javascript object>

In [99]:
# save as json
mmcv.dump(
    data_dict, os.path.join(path_splits, dataset_type + "/r_" + dataset_type + ".json")
)

<IPython.core.display.Javascript object>

In [100]:
df = pd.DataFrame(
    np.array(df_rows),
    columns=[
        "filename",
        "cx",
        "cy",
        "w",
        "h",
        "a",
        "x1",
        "y1",
        "x2",
        "y2",
        "x3",
        "y3",
        "x4",
        "y4",
    ],
)
df.to_csv(os.path.join(path_splits, dataset_type + "/r_" + dataset_type + ".csv"))

<IPython.core.display.Javascript object>